# Tutorial on creating a custom operator (e.g. gate)

This tutorial demonstrates the process of creating your own gate operation.  One can view gate (or layer) operations in pyGSTi as simply parameterized process matrices: a mapping that associates with any given set of parameter values a process matrix.  This mapping is encapsulated by a `LinearOperator`-derived class in pyGSTi, and in addition to using the ones included with pyGSTi (e.g. `FullDenseOp`, see the [Operator tutorial](Operators.ipynb) for more examples) you're free to make your own.  That's exactly what we'll be doing here.

There are lots of good reasons for doing this, the foremost being that you have a specific way you want to model a gate operation that is specific to your system's physics and not captured by pyGSTi's more generic built-in operation classes.  You also may want to make an operation whose parameters are exactly the "knobs" that you have access to in the lab.  Whatever the reason, pyGSTi has been designed to make the creation of custom operator types easy and straightforward.

In this example, we'll be creating a custom 1-qubit gate operation.  It will be a $X(\pi/2)$-rotation that may have some amount of depolarization and "on-axis" overrotation, but no other imperfections.  Thus, it will only have to parameters: the depolarization and the overrotation amounts.

Here's a class which implements this operation.  We'll explain what different parts do below.

In [4]:
import pygsti
import numpy as np

class MyXPiOver2Op(pygsti.obj.DenseOperator):
    def __init__(self):
        self.from_vector([0,0])
        
    def num_params(self): 
        return 2
    
    def to_vector(self):
        return np.array([self.depol_amt, self.over_rotation],'d')
        
    def from_vector(self,v):
        self.depol_amt = v[0]
        self.over_rotation = v[1]
        
        theta = (np.pi/2 + self.over_rotation)/2
        a = 1.0-self.depol_amt
        b = 2*np.cos(theta)*np.sin(theta)
        c = np.sin(theta)**2 - np.cos(theta)**2
        
        self.base = np.array([[1,   0,   0,   0],
                              [0,   a,   0,   0],
                              [0,   0,   c,  -b],
                              [0,   0,   b,   c]],'d')

In [8]:
pygsti.construction.single_qubit_gate(np.pi/4 + 0.05,0,0)

array([[ 1.00000000e+00,  0.00000000e+00,  1.46254688e-16,
         1.71014125e-17],
       [ 0.00000000e+00,  1.00000000e+00,  0.00000000e+00,
         0.00000000e+00],
       [-7.63615546e-17,  0.00000000e+00, -9.98334166e-02,
        -9.95004165e-01],
       [ 1.21168839e-16,  0.00000000e+00,  9.95004165e-01,
        -9.98334166e-02]])

In [44]:
a = (np.pi/4+0.05) 
print(np.sin(a) )
print(np.cos(a) )
print(np.sin(a)**2 )
print(np.cos(a)**2 )
print(np.cos(a)*np.sin(a) )
print(0.5 * 2*(np.sin(a)**2 - np.cos(a)**2) )
print(0.5 * 2*(2*np.cos(a)*np.sin(a)) )
X = np.array([[0,1],[1,0]],'d')

0.7415636913464777
0.6708824723277438
0.5499167083234141
0.4500832916765859
0.49750208263901285
0.09983341664682821
0.9950041652780257


In [37]:
import scipy.linalg as spl
M = pygsti.tools.unitary_to_process_mx(spl.expm( -1j*(np.pi/4+0.05)*X ))
pygsti.tools.print_mx(M)

   0.4501       +0j        0  +0.4975j        0  -0.4975j   0.5499       +0j
        0  +0.4975j   0.4501       +0j   0.5499       +0j        0  -0.4975j
        0  -0.4975j   0.5499       +0j   0.4501       +0j        0  +0.4975j
   0.5499       +0j        0  -0.4975j        0  +0.4975j   0.4501       +0j



In [35]:
basis = pygsti.obj.Basis("pp",2)
S = basis.get_from_std()
pygsti.tools.print_mx(S)

   0.7071       +0j        0       +0j        0       +0j   0.7071       +0j
        0       +0j   0.7071       +0j   0.7071       +0j        0       +0j
        0       +0j        0  +0.7071j        0  -0.7071j        0       +0j
   0.7071       +0j        0       +0j        0       +0j  -0.7071       +0j



In [40]:
np.dot(S,np.dot(M,np.linalg.inv(S)))

array([[ 1.00000000e+00+0.00000000e+00j,  0.00000000e+00-4.76210391e-18j,
         1.46254688e-16+0.00000000e+00j,  1.71014125e-17+0.00000000e+00j],
       [ 0.00000000e+00+4.74049495e-34j,  1.00000000e+00+0.00000000e+00j,
         0.00000000e+00+2.09603730e-17j,  0.00000000e+00-1.07545574e-17j],
       [-7.63615546e-17+0.00000000e+00j,  0.00000000e+00+1.21168839e-16j,
        -9.98334166e-02+0.00000000e+00j, -9.95004165e-01+0.00000000e+00j],
       [ 1.21168839e-16+0.00000000e+00j,  0.00000000e+00-2.98986439e-17j,
         9.95004165e-01+0.00000000e+00j, -9.98334166e-02+0.00000000e+00j]])

In [42]:
pygsti.tools.print_mx(np.linalg.inv(S))

   0.7071       +0j        0       +0j        0       +0j   0.7071       +0j
        0       +0j   0.7071       +0j        0  -0.7071j        0       +0j
        0       +0j   0.7071       +0j        0  +0.7071j        0       +0j
   0.7071       +0j        0       +0j        0       +0j  -0.7071       +0j

